In [ ]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install matplotlib

In [ ]:
import time
import numpy as np
import pandas as pd
from keras.callbacks import TensorBoard
from keras.layers import Dense
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.callbacks import Callback
from itertools import groupby
from statistics import mean
import matplotlib.pyplot as plt
from datetime import timedelta

In [ ]:
def plot_training_summary(training_summary, time_summary=None):
    if time_summary:
        print('Training time: ' + str(timedelta(seconds=time_summary.training_time)))
        print('Epoch time avg: '  + str(timedelta(seconds=mean(time_summary.epoch_times))))
    hist = sorted(training_summary.history.items(),
                  key=lambda x: (x[0].replace('val_', ''), x[0]))

    epochs = [e + 1 for e in training_summary.epoch]
    for metric, values in groupby(hist,
                                  key=lambda x: x[0].replace('val_', '')):
        if 'val_loss' in training_summary.history:
            val0, val1 = tuple(values)
            plt.plot(epochs, val0[1], epochs, val1[1], '--', marker='o')
        else:
            val0 = tuple(values)[0]
            plt.plot(epochs, val0[1], '--', marker='o')
        plt.xlabel('epoch'), plt.ylabel(val0[0])
        plt.legend(('Train set', 'Validation set'))
        plt.show()

In [ ]:
class TimeSummary(Callback):
    def on_train_begin(self, logs={}):
        self.epoch_times = []
        self.training_time = time.process_time()

    def on_train_end(self, logs={}):
        self.training_time = time.process_time() - self.training_time

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.process_time()

    def on_epoch_end(self, batch, logs={}):
        self.epoch_times.append(time.process_time() - self.epoch_time_start)

In [ ]:
class NNKeras:
    def __init__(self, url: str):
        self._url = url
        self._num_cols = 64
        self._call_back = TensorBoard(log_dir='../logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

    def read_data(self):
        df = pd.read_csv(self._url, header=None)
        X = df.iloc[:, 1:].astype(float)
        classes = df.iloc[:, 0]
        unique_classes = pd.DataFrame(sorted([c.upper() for c in classes.unique()]))
        rows = X.shape[0]
        unique_classes['indices'] = range(1, len(unique_classes) + 1)
        y = np.zeros((rows, len(unique_classes)), np.bool)
        for i in range(rows):
            col_idx = np.where(unique_classes.loc[:, 0] == classes[i].upper())
            y[i, col_idx] = True
        return X, y, unique_classes

    def base_model(self, nodes):
        model = Sequential()

        for prev_node, node in zip(nodes[:-1], nodes[1:]):
            model.add(Dense(node, activation='relu', input_dim=prev_node))  # Add the first hidden layer

        model.add(Dense(31, activation='sigmoid'))  # Add the output layer
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model

    def train(self, X, y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=5)
        time_summary = TimeSummary()
        for num_nodes in range(2, 100):
            nodes = [64, num_nodes]
            model = self.base_model(nodes)
            summary = model.fit(X_train, y_train, epochs=10, verbose=0, callbacks=[self._call_back,time_summary])
            score = model.evaluate(X_test, y_test)
            plot_training_summary(summary, time_summary)
            print('Test loss:', score[0])
            print('Test accuracy:', score[1])

In [ ]:
nn = NNKeras("https://raw.githubusercontent.com/hardikgw/neural-network-poc/master/data/dataset/dataset.csv")

In [11]:
X, y, unique_classes = nn.read_data()

In [13]:
nn.train(X, y)

157/157 [==============================] - 2s 11ms/step
2 0.936305732104429
157/157 [==============================] - 2s 11ms/step
3 0.9677418956331386
157/157 [==============================] - 2s 11ms/step
4 0.925826960688184
157/157 [==============================] - 2s 16ms/step
5 0.9369221337281974
157/157 [==============================] - 2s 14ms/step
6 0.9465790041692698
157/157 [==============================] - 3s 16ms/step
7 0.9387713199967791
157/157 [==============================] - 3s 17ms/step
8 0.9677418956331386
157/157 [==============================] - 3s 17ms/step
9 0.9677418956331386
157/157 [==============================] - 5s 32ms/step
10 0.9677418956331386
157/157 [==============================] - 4s 22ms/step
11 0.9383604055756976
157/157 [==============================] - 4s 23ms/step
12 0.9677418956331386
157/157 [==============================] - 4s 26ms/step
13 0.9677418956331386
157/157 [==============================] - 2s 14ms/step
14 0.9677418956331

KeyboardInterrupt: 